<a href="https://colab.research.google.com/github/ShreyPatel1311/Quantum-Computing/blob/main/Auto-Tune%20Charge%20State%20for%20Quantum%20Dots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://gitlab.com/QMAI/mlqe_2023_edx.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy as sp
from scipy.special import softmax
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import glob
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
with open('mlqe_2023_edx/week3/dataset/csds.npy', 'rb') as f:
    data = np.load(f)
with open('mlqe_2023_edx/week3/dataset/labels.npy', 'rb') as f:
    labels = np.load(f)

In [ ]:
data.shape

In [ ]:
number_of_data = 2000
data = data[:number_of_data]
labels = labels[:number_of_data]

In [ ]:
fig, ax = plt.subplots(1, 10, figsize = (20,10))
for index, d in enumerate(data[np.random.choice(len(data), size = 10)]):
    ax[index].imshow(data[index])
    ax[index].axis('off')
    ax[index].set_title(f'Label: {(labels[index])}')
plt.show()
plt.close()

In [ ]:
class CreateDataset(Dataset):
  def __init__(self, data, labels):
    self.data = torch.Tensor(data)
    self.labels = torch.Tensor(labels)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
        data_idx = self.data[idx]
        label = self.labels[idx].type(torch.LongTensor)
        return data_idx, label

In [ ]:
dataset = CreateDataset(data, labels)
train_set, test_set = torch.utils.data.random_split(dataset, (int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)))

In [ ]:
batch_size = 8
trainloader = DataLoader(train_set, batch_size = batch_size)
testloader = DataLoader(test_set, batch_size = batch_size)

In [ ]:
for X, y in trainloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    print(y)
    break

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_stack = nn.Sequential(
        nn.Linear(50 * 50, 100),
        nn.GELU(),
        nn.Linear(100, 16),
        nn.ReLU(),
        nn.Linear(16, 2)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_stack(x)
    return logits

In [ ]:
model = NeuralNetwork().to(device)
lossFN = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
train_loss_dnn = []
train_acc_dnn = []
test_loss_dnn = []
test_acc_dnn = []

In [ ]:
def train(dataloader, model, lossFN, optimizer, train_loss, train_acc):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.train()

  running_loss, correct = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = lossFN(pred, y)
    train_loss.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    if(batch % batch_size == 0):
      loss,current = loss.item(), batch*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

  running_loss /= num_batches
  correct /= size

  train_acc.append(correct)
  train_loss.append(running_loss)

def test(dataloader,model,loss_fn,test_loss, test_acc):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  running_loss, correct = 0, 0
  with torch.no_grad():
      for X,y in dataloader:
          X,y = X.to(device), y.to(device)
          pred = model(X)
          running_loss += loss_fn(pred, y).item()
          correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  running_loss /= num_batches
  correct /= size

  test_acc.append(correct)
  test_loss.append(running_loss)
  print(f"Test Error: \n Accuracy {(100*correct):>0.1f}%, Avg loss:{running_loss:>8f}\n")

In [ ]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n -------------------")
    train(trainloader,model,lossFN,optimizer, train_loss_dnn, train_acc_dnn)
    test(testloader,model,lossFN, test_loss_dnn, test_acc_dnn)
print("Done!")

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv_1st = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1)
    self.conv_2nd = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1)
    self.conv_3rd = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, stride=1)
    self.relu = nn.ReLU()
    self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.flatten = nn.Flatten()
    self.seq = nn.Sequential(
        nn.Linear(in_features=64 * 2 * 2, out_features=64),
        nn.ReLU(),
        nn.Linear(in_features=64, out_features=32),
        nn.ReLU(),
        nn.Linear(in_features=32, out_features=2),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
    x = x.view(-1, 1, 50, 50)
    x = self.conv_1st(x)
    x = self.relu(x)
    x = self.max_pool(x)
    x = self.conv_2nd(x)
    x = self.relu(x)
    x = self.max_pool(x)
    x = self.conv_3rd(x)
    x = self.relu(x)
    x = self.max_pool(x)
    x = self.flatten(x)
    x = self.seq(x)
    return x

In [ ]:
train_loss_cnn = []
train_acc_cnn = []
test_loss_cnn = []
test_acc_cnn = []

In [ ]:
num_epochs = 15
learning_rate = 1e-4

model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}\n -------------------")
  train(trainloader, model, criterion, optimizer, train_loss_cnn, train_acc_cnn)
  test(testloader, model, criterion, test_loss_cnn, test_acc_cnn)


In [ ]:
def create_acc_loss_graph(train_acc,train_loss, test_acc, test_loss):
    fig, axes = plt.subplots(ncols=2, nrows=1, dpi=300)
    fig.set_size_inches(9, 3)
    ax1, ax2 = axes[0], axes[1]

    ax1.plot(train_acc,'-o',label="train", markersize=4)
    ax1.plot(test_acc,'--+',label="test", markersize=4)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc=3)

    ax2.plot(train_loss,'-o',label="train", markersize=4)
    ax2.plot(test_loss,'--+',label=" test", markersize=4)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc=1)

    #plot parameters
    ax1.set_ylim(0, np.max([np.max(train_acc),np.max(test_acc)])+0.1)
    ax2.set_ylim(-0.1, np.max([np.max(train_loss),np.max(test_loss)])+0.1)
    ax1.grid(True, which='both',linewidth=0.1)
    ax2.grid(True, which='both',linewidth=0.1)
    plt.tight_layout()
    plt.show()

In [ ]:
create_acc_loss_graph(train_acc_dnn,train_loss_dnn, test_acc_dnn, test_loss_dnn)
create_acc_loss_graph(train_acc_cnn,train_loss_cnn, test_acc_cnn, test_acc_cnn)